In [55]:
import os
import dotenv

dotenv.load_dotenv()

'123'

In [52]:
import langchain as lc
from langchain.llms import HuggingFaceHub
from langchain.llms import OpenAI

llm = OpenAI()

llm = HuggingFaceHub(
    repo_id="google/flan-t5-xxl", 
    model_kwargs={"temperature": 0.6, "max_length":4096}
)

/home/vagozino/.conda/envs/llm/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [46]:
import glob
import os
from pathlib import Path
from typing import Any, List

from langchain.docstore.document import Document
from langchain.document_loaders import SRTLoader
from langchain.document_loaders.base import BaseLoader


class LectureLoader(BaseLoader):
    def __init__(self, 
                 lecture_file: str,
                 add_lecture_info: bool = False
                ):
        self.add_lecture_info = add_lecture_info
        self.lecture_file = lecture_file

    @classmethod
    def from_folder(cls, folder_name: str, **kwargs: Any) -> 'LectureLoader':
        return cls(folder_name, kwargs)

    def load(self) -> List[Document]:
        documents = []
        
        for file_name in Path(self.lecture_file).rglob('*'):
            
            file_path = Path(file_name)
            if not file_path.is_file(): continue
                
            with open(file_name, "r") as f:

                metadata = {}
                
                # Load the transcript data
                if file_name.suffix == ".srt":
                # or file_name.suffix == ".sbv" \
                # or file_name.suffix == ".vtt" \
                # or file_name.suffix == ".txt":

                    srt_loader = SRTLoader(file_name)
                        
                    if self.add_lecture_info:
                        metadata["lecture_name"] = file_path.parent.name
                        metadata["source"] = file_path.stem
                        metadata["type"] = "transcript"

                    for doc in srt_loader.load():
                        doc.metadata.update(metadata)
                        documents.append(doc)

        return documents

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores.base import VectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.embeddings import Embeddings

from typing import Type, Iterable, Optional, List


class LectureIndex(FAISS):
    """Wrapper around the FAISS VectorStore"""

    def from_documents(documents: List[Document], embedding: Embeddings):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = 700, chunk_overlap = 0)
        docs_split = text_splitter.split_documents(documents)
        return FAISS.from_documents(docs_split, embedding)

In [47]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

lecture_loader = LectureLoader.from_folder("./data", add_lecture_info=True)
lecture_docs = lecture_loader.load()
print(lecture_docs)

lecture_index = LectureIndex.from_documents(lecture_docs, embeddings)

[Document(page_content='Professor Steven Smith:\nThere is one person in here, I don\'t know who it is,\nand you will not know who it is yet, but there is one person in\nhere for whom the reading of Plato\'s Republic will be\nthe most important intellectual experience you have at Yale. It is a book that one of you\nwill go back to time and time again and it will stick with you\nforever. What I would like you to do is\nto remember this and four years from now, when most of you are\nready to graduate, if that one person in here\nwould email me and let me know who it is, okay? Maybe it will be you.\nMaybe? Possibly.\nOr you. Okay.\nThis is the book that started it all.\nThe Apology, the Crito,\nthese are warm-ups to the big theme, to the big book,\nthe Republic. Every other book in this\npolitical science that has since been written,\nbeginning with Aristotle\'s Politics and moving on to\nthe present day is, in one way or another,\nan answer, a response to Plato\'s Republic.\nIt started th

In [50]:
results = lecture_index.similarity_search("Student?")

for doc in results:
    print("- ", doc.metadata)

-  {'source': "4. Philosophers and Kings： Plato's Republic, I-II [nVQKbQVc2_w].en", 'lecture_name': 'lecture1', 'type': 'transcript'}
-  {'source': "4. Philosophers and Kings： Plato's Republic, I-II [nVQKbQVc2_w].en", 'lecture_name': 'lecture1', 'type': 'transcript'}
-  {'source': "4. Philosophers and Kings： Plato's Republic, I-II [nVQKbQVc2_w].en", 'lecture_name': 'lecture1', 'type': 'transcript'}
-  {'source': "4. Philosophers and Kings： Plato's Republic, I-II [nVQKbQVc2_w].en", 'lecture_name': 'lecture1', 'type': 'transcript'}
